In [ ]:
import numpy as np
import sympy as sp
from typing import List
import numpy.linalg as la
import scipy.optimize as opt

In [ ]:
# Golden Section Search 1D
def golden_single(f,x:List[sp.core.symbol.Symbol],a:float,b:float,tol=0,tau=(np.sqrt(5) - 1)/2, iteration=-1):
  h = (b-a)
  x1 = a + ((1-tau)*h)
  x2 = a + (tau*h)
  f1 = f.subs(x[0],x1)
  f2 = f.subs(x[0],x2)
  num = 0
  while (h>=tol or (tol==0 and iteration!=-1)):
    h = tau*h
    if (f1<f2):
      b = x2
      x2 = x1
      f2 = f1
      x1 = a+((1-tau)*h)
      f1 = f.subs(x[0],x1)
    else:
      a = x1
      x1 = x2
      f1 = f2
      x2 = a + tau*h
      f2 = f.subs(x[0],x2)
    num += 1
    if(iteration!=-1 and num>=iteration): break
  return (a,b)

# calculate the interval length after given number of iteration
def get_interval_len(a:float,b:float,iteration:int, tau=(np.sqrt(5) - 1)/2):
  return (tau**iteration)*(b-a)


In [ ]:
# 1D Newton's method optimization
# (Find Min)
def Newton_single(f,x:List[sp.core.symbol.Symbol],x0:np.ndarray,iteration=-1):
  df1 = f.diff(x[0],1).subs(x[0],x0[0])
  df2 = f.diff(x[0],2).subs(x[0],x0[0])
  h = df1/df2
  x1 = x0[0]
  num = 0
  while(h!=0 and (num<iteration or iteration==-1)):
    x1 = x1-h
    df1 = f.diff(x[0],1).subs(x[0],x1)
    df2 = f.diff(x[0],2).subs(x[0],x1)
    h = df1/df2
    num += 1 
  return (x1,num)

# print the approximation function in newton method
def Newton_function_approx(f,x:List[sp.core.symbol.Symbol],x0:np.ndarray):
  # f_approx = f0 + df1*(x-x0[0])+ 0.5*df2*(x-x0[0])^2
  # sometimes, (x-x0[0]) = h
  f0 = f.subs(x[0],x0[0])
  df1 = f.diff(x[0],1).subs(x[0],x0[0])
  df2 = f.diff(x[0],2).subs(x[0],x0[0])
  print(f0)
  print(str(df1)+str("(x-x0[0])"))
  print(str(0.5*df2)+str("(x-x0[0])^2"))

x = sp.symbols("x:1")
f = 4*x[0]**3+2*x[0]**2+5*x[0]+40
Newton_single(f,x,[2],1)

(43/52, 1)

In [ ]:
# ND optimization
# hessian[i][j] = ∂^2(f)/∂(xi)∂(xj)
def Hessian_matrix(f,x:List[sp.core.symbol.Symbol]):
  hessian = []
  for i in range(len(x)):
    hessian.append([])
    for j in range(len(x)):
      hessian[i].append(f)
  hessian = np.array(hessian)
  for i in range(len(x)):
    for j in range(i,len(x)):
      ddf = hessian[i][j].diff(x[i],1).diff(x[j],1)
      hessian[i][j] = ddf
      hessian[j][i] = ddf
  return hessian

# return ∇f
def f_grad(f,x:List[sp.core.symbol.Symbol]):
  delta_f = []
  for i in range(len(x)):
    delta_f.append(f)
  for i in range(len(x)):
      delta_f[i] = delta_f[i].diff(x[i],1)
  return np.array(delta_f)

# Solve hessian matrix and get H(x0)
# Return the value of H(x0), hessian(x0)
def Hx(hessian, x:List[sp.core.symbol.Symbol], x0:np.ndarray):
  h = np.zeros(hessian.shape)
  for i in range(len(x)):
    for j in range(i,len(x)):
      ddf = hessian[i][j]
      for k in range(len(x)):
        ddf = ddf.subs(x[k],x0[k])
      h[i][j] = ddf
      h[j][i] = ddf 
  return h

# return ∇f(x0)
# delta_f = f_grad(f,x)
def solve_f_grad(delta_f,x:List[sp.core.symbol.Symbol],x0:np.ndarray):
  dfx = np.zeros(delta_f.shape)
  for i in range(len(x)):
    df = delta_f[i]
    for k in range(x0.shape[0]):
      df = df.subs(x[k],x0[k])
    dfx[i] = df 
  return dfx

# Perform ND optimal solution
# First order: find critical points
# Second order: check the type of critical point by using eigenvals
# Will print out all critical points and its type
def solve_ND_optimal(f,x:List[sp.core.symbol.Symbol]):
  delta_f = f_grad(f,x)
  hessian = Hessian_matrix(f,x)
  stat_points = sp.solve(delta_f,x)
  for xi in stat_points:
    hi = Hx(hessian, x, xi)
    eig = la.eig(hi)
    pos = 0
    neg = 0
    for i in eig[0]:
      if (i>0): pos += 1
      else: neg += 1
    if (neg!=0 and pos!=0):   print("Saddle Point: "+str(xi))
    elif (neg==0 and pos!=0): print("Minimum     : "+str(xi))
    elif (neg!=0 and pos==0): print("Maximum     : "+str(xi))
    else: print("Error")
  
x = sp.symbols("x:2")
f = 3+ (x[0]**2)*x[1]/8+(x[1]**2)/8
x0 = np.array([np.pi/3,np.pi/(2*np.sqrt(2))])
solve_ND_optimal(f,x)
delta_f = f_grad(f,x)
dfx = solve_f_grad(delta_f,x,x0)
print(dfx)
hessian = Hessian_matrix(f,x)
hx = Hx(hessian, x, x0)
print(hx)

Saddle Point: (0, 0)
[0.29078601 0.41475802]
[[0.27768018 0.26179939]
 [0.26179939 0.25      ]]


In [ ]:
# Steepest Descent Method

# function wrapper for line search
def func(alpha, f,x:List[sp.core.symbol.Symbol],x0:np.ndarray):
  df  = f_grad(f,x)
  dfx = solve_f_grad(delta_f,x,x0)
  xk = np.float64(x0-alpha*dfx)
  val = f
  for i in range(len(x)):
    val = val.subs(x[i],xk[i])
  val = np.float64(val)
  return val


# If alpha was given, or learning rate not given (alpha==1), then
# use this method, and set alpha_ini to the given alpha
# if alpha_ini==0, the function will use line search to find optimized alpha
def steepest_descent_alpha(f,x:List[sp.core.symbol.Symbol],x0:np.ndarray,tol,alpha_ini=1,iteration=-1):
  delta_f = f_grad(f,x)
  xk = np.float64(x0)
  num = 0
  s_list = []
  a_list = []
  alpha = alpha_ini
  while(num<iteration or iteration==-1):
    if(alpha_ini==0): alpha = opt.golden(func, args = (f,x,xk))
    dfx = solve_f_grad(delta_f,x,xk)
    sk = -1*alpha*dfx
    if(la.norm(sk, np.inf)<tol): break
    s_list.append(sk)
    a_list.append(alpha)
    xk = xk + sk
    num += 1
  return (xk,s_list,num)


# use this only when learning rate was given
def steepest_descent_learning_rate(f,x:List[sp.core.symbol.Symbol],x0:np.ndarray,learning_rate,tol,iteration=-1):
  delta_f = f_grad(f,x)
  xk = np.float64(x0)
  num = 0
  s_list = []
  while(num<iteration or iteration==-1):
    dfx = solve_f_grad(delta_f,x,xk)
    sk = -1*learning_rate*dfx
    if(la.norm(sk, np.inf)<tol): break
    s_list.append(sk)
    xk = xk + sk
    num += 1
  return (xk,s_list,num)


In [ ]:
# ND Newton's method (Find Min)
def ND_Newton(f,x:List[sp.core.symbol.Symbol],x0:np.ndarray,tol, iteration=-1):
  num = 0
  hessian = Hessian_matrix(f,x)
  delta_f = f_grad(f,x)
  xk = x0
  while(num<iteration or iteration==-1):
    hx = Hx(hessian, x, xk)
    dfx = solve_f_grad(delta_f,x,xk)
    sk = la.solve(hx,-1*dfx)
    if(la.norm(sk, np.inf)<tol): break
    xk = xk+sk
    num += 1
  return (xk, sk, num)
  
x = sp.symbols("x:2")
f = -1*(sp.exp(-1*(x[0]**2)))*(x[0]+sp.sin(x[0]))
print(f.diff(x[0],2))

(4*x0*(cos(x0) + 1) - 2*(x0 + sin(x0))*(2*x0**2 - 1) + sin(x0))*exp(-x0**2)


In [ ]:
x = sp.symbols("x:1")
f = 3*(x[0]**3) + x[0]**2 + 3*x[0] + 1
x0 = np.array([2])
Newton_function_approx(f,x,x0)

35
43(x-x0[0])
19.0000000000000(x-x0[0])^2


In [ ]:
x = sp.symbols("x:2")
x0 = np.array([np.pi/3,np.pi/(2*np.sqrt(2))])
f = 3+((x[0]**2)/8)+((x[1]**2)/8)-sp.sin(x[0])*sp.cos(x[1]*sp.sqrt(2)/2)
print(f)
print(solve_f_grad(f_grad(f,x),x,x0))
print(Hx(Hessian_matrix(f,x), x, x0))
print(ND_Newton(f,x,x0,10**-8, iteration=1))
print(steepest_descent_alpha(f,x,x0,10**-8,alpha_ini=1.581, iteration=1))

x0**2/8 + x1**2/8 - sin(x0)*cos(sqrt(2)*x1/2) + 3
[-0.091754    0.71069289]
[[0.86237244 0.25      ]
 [0.25       0.55618622]]
(array([ 1.59546844, -0.41351805]), array([ 0.54827089, -1.52423879]), 1)
(array([ 1.19226063, -0.01288472]), [array([ 0.14506308, -1.12360545])], 1)


In [ ]:
x = sp.symbols("x:2")
x0 = np.array([0,1])
f = 2*x[0]**2+2*x[0]*x[1]+2*x[1]**2+9*sp.exp(9*x[0]*x[1])+7*((sp.sin(x[1]))**2)+5*sp.cos(x[0]*x[1])
print(f)
print(solve_f_grad(f_grad(f,x),x,x0))
print(Hx(Hessian_matrix(f,x), x, x0))
print(ND_Newton(f,x,x0,10**-8, iteration=1))
print(steepest_descent_alpha(f,x,x0,10**-8,alpha_ini=1, iteration=1))

2*x0**2 + 2*x0*x1 + 2*x1**2 + 9*exp(9*x0*x1) + 7*sin(x1)**2 + 5*cos(x0*x1)
[83.         10.36508199]
[[728.          83.        ]
 [ 83.          -1.82605571]]
(array([-0.1231223 ,  1.07991607]), array([-0.1231223 ,  0.07991607]), 1)
(array([-83.        ,  -9.36508199]), [array([-83.        , -10.36508199])], 1)


In [ ]:
# N-Dimension Optimization using Steepest Descent
x = sp.symbols("x:2")
x0 = np.array([-2,2])
f = 4*x[0]**2+13*x[0]*x[1]+4*(x[1]**2)+13*(sp.sin(x[1])**2)+9*sp.cos(x[0]*x[1])
steepest_descent_alpha(f,x,x0,10**-8,alpha_ini=0.05,iteration=1)

(array([-1.81887775,  2.31079938]), [array([0.18112225, 0.31079938])], 1)

In [ ]:
# Perform One Step of Golden Section Search
x = sp.symbols("x:1")
f = (x[0]-6.2)**4
golden_single(f,x,-6,8,iteration=1)

(-0.6524758424985286, 8)

In [ ]:
# Carrying out Newton steps
x = sp.symbols("x:1")
x0 = np.array([0.35])
f = 5*(sp.cos(x[0])**5)
print(Newton_single(f,x,x0,iteration=1))

(-0.431617494708300, 1)


In [ ]:
# Carrying out Newton steps (n-dimensional)
x = sp.symbols("x:2")
x0 = np.array([0,np.pi])
f = sp.exp(4*x[0])+7*sp.cos(x[1])
ND_Newton(f,x,x0,10**-8, iteration=1)

(array([-0.25      ,  3.14159265]), array([-2.5000000e-01,  1.2246468e-16]), 1)

In [ ]:
# Carrying out Newton steps
x = sp.symbols("x:1")
x0 = np.array([0.45])
f = sp.cos(x[0])
Newton_single(f,x,x0,iteration=1)

(-0.0330550656165784, 1)

In [ ]:
# Golden Section Interval
x = sp.symbols("x:1")
x0 = np.array([0.45])
f = (x[0]-7.2)**4
golden_single(f,x,-6,13,tol=10**-8,tau=(np.sqrt(5) - 1)/2, iteration=1)

(1.2573542137519969, 13)

In [ ]:
# Optimization ND NewtonMethod one step
x = sp.symbols("x:2")
x0 = np.array([1,3])
f = (4*x[0]**3)+2*(x[1]**2)
print(Hx(Hessian_matrix(f,x), x, x0))
print(ND_Newton(f,x,x0,10**-8, iteration=1))

[[24.  0.]
 [ 0.  4.]]
(array([0.5, 0. ]), array([-0.5, -3. ]), 1)


In [ ]:
# Determine the length of the interval after one iteration
x = sp.symbols("x:1")
x0 = np.array([0.45])
f = (x[0]-7.2)**4
get_interval_len(-12,10,1, tau=(np.sqrt(5) - 1)/2)

13.596747752497688

In [ ]:
# Carrying out Newton steps
x = sp.symbols("x:1")
x0 = np.array([0.35])
f = -1*sp.exp(-1*(x[0]**2))
Newton_single(f,x,x0,iteration=1)

(-0.113576158940397, 1)

In [ ]:
x = sp.symbols("x:2")
x0 = np.array([-5,5])
f = 10*x[0]**2 + 11*x[0]*x[1]+10*x[1]**2+2*(sp.sin(x[1])**2)+12*sp.cos(x[0]*x[1])
print(steepest_descent_alpha(f,x,x0,10**-8,alpha_ini=0.3,iteration = 1))
print(steepest_descent_learning_rate(f,x,x0,0.3,10**-8,iteration = 1))

(array([ 10.8823315 , -10.55591884]), [array([ 15.8823315 , -15.55591884])], 1)
(array([ 10.8823315 , -10.55591884]), [array([ 15.8823315 , -15.55591884])], 1)


In [ ]:
x = sp.symbols("x:1")
x0 = np.array([0.4])
f = 4*sp.sin(x[0]**5)
Newton_single(f,x,x0,iteration = 1)

(0.299986890623519, 1)

In [ ]:
x = sp.symbols("x:1")
f = (x[0]-6.7)**2
golden_single(f,x,-5,12,iteration =1)

(1.4934221912517867, 12)

In [ ]:
# N-Dimension Optimization using Steepest Descent
x = sp.symbols("x:2")
x0 = np.array([-2,5])
f = 13*(x[0]**2)+4*x[0]*x[1]+13*x[1]**2+12*(sp.sin(x[1])**2)+13*sp.cos(x[0]*x[1])
steepest_descent_alpha(f,x,x0,10**-8,iteration = 1)

(array([  84.63986127, -161.71183346]),
 [array([  86.63986127, -166.71183346])],
 1)

In [ ]:
x = sp.symbols("x:2")
x0 = np.array([1,3])
f = 6*x[0]**4+3*x[1]**2

hessian = Hessian_matrix(f,x)
ND_Newton(f,x,x0,tol=10**-12,iteration=1)

(array([0.66666667, 0.        ]), array([-0.33333333, -3.        ]), 1)